In [1]:
import pandas as pd
import numpy as np
import os 

images = os.listdir("./images")

In [2]:
keys = pd.read_csv("numbers_for_image.csv", header=None, names=['image','numbers'])

keys['numbers'] = keys['numbers'].apply(lambda x: x.replace("[", ""))
keys['numbers'] =keys['numbers'].apply(lambda x: x.replace("]",""))
keys['numbers'] =keys['numbers'].apply(lambda x: x.split(","))

keys['numbers'] = keys['numbers'].apply(lambda x: np.setdiff1d(x, [""]))

empty_photos = keys[keys['numbers'].apply(lambda x: len(x) == 0)]

In [3]:
one_number_pictures = keys[keys['numbers'].apply(lambda x: len(x)) == 1]
one_number_pictures['numbers'] = one_number_pictures['numbers'].apply(lambda x: int(x[0].replace("'","")))

/home/egert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [3]:
useful_photos =  keys[~keys['numbers'].apply(lambda x: len(x) == 0)]

In [22]:
from PIL import Image

for i in empty_photos['image'].tail(10):
    img = Image.open('./images/' + str(i)+'.jpg')
    img.show()


In [10]:
import cv2

# Get user supplied values
imagePath = "~/Desktop/Number_reader_from_sportfoto/images/3864141.jpg"
cascPath = "/home/egert/Desktop/Number_reader_from_sportfoto/haarcascade_frontalface_default.xml"

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)

In [67]:
# Read the image
image = cv2.imread(imagePath,0)
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [74]:
# Detect faces in the image
faces = faceCascade.detectMultiScale(
    image,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.cv.CV_HAAR_SCALE_IMAGE
)

AttributeError: module 'cv2.cv2' has no attribute 'cv'

In [25]:
for image in useful_photos['image']:
    print(image)
    break

3928792


In [143]:
import cv2
from PIL import Image

picture_info = {}

for image in one_number_pictures['image']:

    path ="./images/"+ str(image)+".jpg"
    original_image = cv2.imread(path,0)

    cascPath = "haarcascade_frontalface_default.xml"

    face_cascade = cv2.CascadeClassifier(cascPath)
    detected_faces = face_cascade.detectMultiScale(original_image)
    
    if(len(detected_faces) == 1):
        picture_info[image] = detected_faces[0]
    
#         for (column, row, width, height) in detected_faces:
#             cv2.rectangle(
#                 original_image,
#                 (column, row),
#                 (column + width, row + height),
#                 (0, 255, 0),
#                 2
#         )
#         img = Image.fromarray(original_image)
#         img.show()

In [8]:
import pandas as pd
data = pd.read_csv("images_data.csv")

In [6]:
data.head()

,image,left,top,bottom,right,numbers
0,3863680,138,190,490,438,108
1,3863685,151,183,483,451,186
2,3863686,157,185,485,457,186
3,3863694,123,194,494,423,649
4,3863699,161,230,530,461,553


In [16]:
import cv2
from PIL import Image
for index, row in data.iloc[:30].iterrows():
    path ="./images/"+ str(row['image'])+".jpg"
    img = cv2.imread(path,0)

    cv2.rectangle(
        img,
        (row['top'], row['left']),
        (row['bottom'], row['right']),
        (0, 255, 0),
        2
        )

    
#     cv2.rectangle(
#         img,
#         (row['column'], row['row']),
#         (row['column'] + row['width'], row['row'] + row['height']),
#         (0, 255, 0),
#         2
#     )
    img = Image.fromarray(img)
#     img.show()
    
    img.save("./Example_photos/" + str(row['image']) + ".jpg")
#     img = img.crop((row['column'] -100,row['row'] + 100,row['column'] -100 + width, row['row']+100 + height))
#     img.show()
#     print(index, row)


In [144]:
data =pd.DataFrame.from_dict(picture_info, orient='index', columns=['column', 'row', 'width', 'height'])


data['left'] =  data[['column','row']].apply(lambda x: x['column'] - 100, axis=1)

data['top'] =  data[['column','row']].apply(lambda x: x['row'] + 100, axis=1)


data['bottom'] = data[['column','row']].apply(lambda x: x['row'] + 100 +300, axis=1)
data['right'] = data[['column','row']].apply(lambda x: x['column'] - 100 + 300 , axis=1)


data = data.drop(columns=['column', 'row', 'width', 'height'])
data = data.reset_index()
data = data.rename(columns={'index':'image'})
data = data.set_index('image').join(one_number_pictures.set_index('image'))
data.to_csv('images_data.csv')

In [172]:
from shutil import copyfile
import os

src = "./images"
target = './cloned_project/Multi-Digit-Recognition/toy_data/train'
def copy_images(df, src, target):
    for index, row in df.iterrows():
        copyfile(os.path.join(src, str(index) + '.jpg'), os.path.join(target,str(index) + '.jpg' ))
        
# copy_images(data, src, target)

## Make validation, test, train sets

In [163]:
size = data.shape[0]

train = np.random.choice(data.index, int(size*0.8), replace=False)

split = np.setdiff1d(data.index, train)

validation = np.random.choice(split, int(size*0.15), replace=False)

test = np.setdiff1d(split, validation)

In [173]:
folders = ['./cloned_project/Multi-Digit-Recognition/toy_data/train',
           './cloned_project/Multi-Digit-Recognition/toy_data/test',
          './cloned_project/Multi-Digit-Recognition/toy_data/extra']

dfs = [data.loc[train], data.loc[test], data.loc[validation]]

for folder_path, df in zip(folders, dfs):
    copy_images(df, src, folder_path)